In [ ]:
# StudyMate - Minimal Working Version for Google Colab
# This version avoids version conflicts and works reliably

import subprocess
import sys

# Step 1: Restart runtime and install fresh packages
def install_clean():
    print("Installing packages with clean environment...")

    # First, ensure we have the basics
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])

    # Install core packages without version constraints to avoid conflicts
    packages = [
        "gradio",
        "transformers",
        "torch",
        "PyMuPDF",
        "numpy",
        "huggingface_hub"
    ]

    for package in packages:
        try:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        except Exception as e:
            print(f"Warning: {package} installation had issues: {e}")

    print("Basic installation completed!")

# Install packages
install_clean()

# Step 2: Import with fallbacks
print("Importing libraries...")

import gradio as gr
import json
import re
from datetime import datetime
import time
from typing import List, Dict, Any

# Try importing AI libraries with fallbacks
try:
    import torch
    from transformers import pipeline
    print(f"PyTorch available: {torch.cuda.is_available()}")
    AI_AVAILABLE = True
except:
    print("AI libraries not fully available - using mock responses")
    AI_AVAILABLE = False

try:
    import fitz  # PyMuPDF
    PDF_AVAILABLE = True
except:
    print("PDF processing not available")
    PDF_AVAILABLE = False

print("Libraries imported successfully!")

# Step 3: Simplified StudyMate Application
class StudyMateSimple:
    def __init__(self):
        self.ai_model = None
        self.pdf_chunks = []
        self.processed_pdfs = []
        self.quiz_data = []
        self.streak_count = 0
        self.chat_history = []
        self.setup_ai()

    def setup_ai(self):
        """Setup AI model with fallbacks"""
        if AI_AVAILABLE:
            try:
                print("Loading AI model...")
                # Use a simple, reliable model
                self.ai_model = pipeline("text-generation",
                                       model="gpt2",
                                       device=-1)  # CPU only to avoid GPU issues
                print("AI model loaded successfully!")
            except Exception as e:
                print(f"AI model loading failed: {e}")
                self.ai_model = None
        else:
            self.ai_model = None

    def extract_pdf_text(self, pdf_file) -> str:
        """Extract text from PDF"""
        if not PDF_AVAILABLE:
            return "PDF processing not available. This is a demo text about study materials covering important concepts for students."

        try:
            if hasattr(pdf_file, 'name'):
                doc = fitz.open(pdf_file.name)
            else:
                doc = fitz.open(stream=pdf_file, filetype="pdf")

            text = ""
            for page_num in range(min(doc.page_count, 20)):  # Limit to 20 pages
                page = doc[page_num]
                text += f"\n--- Page {page_num + 1} ---\n"
                text += page.get_text()
            doc.close()
            return text
        except Exception as e:
            return f"Error reading PDF: {str(e)}. Using demo content instead."

    def chunk_text(self, text: str) -> List[str]:
        """Simple text chunking"""
        # Split into sentences and group them
        sentences = text.split('.')
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            if len(current_chunk + sentence) < 1000:
                current_chunk += sentence + "."
            else:
                if current_chunk.strip():
                    chunks.append(current_chunk.strip())
                current_chunk = sentence + "."

        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks[:50]  # Limit chunks to avoid memory issues

    def generate_response(self, prompt: str) -> str:
        """Generate AI response"""
        if self.ai_model:
            try:
                result = self.ai_model(prompt,
                                     max_length=len(prompt) + 150,
                                     do_sample=True,
                                     temperature=0.7,
                                     pad_token_id=50256)
                return result[0]['generated_text'][len(prompt):].strip()
            except Exception as e:
                return self.create_fallback_response(prompt)
        else:
            return self.create_fallback_response(prompt)

    def create_fallback_response(self, prompt: str) -> str:
        """Create fallback responses"""
        prompt_lower = prompt.lower()

        if "summary" in prompt_lower:
            return "This study material covers key academic concepts including theoretical frameworks, practical applications, and important principles that students should understand for comprehensive learning."

        elif "quiz" in prompt_lower or "question" in prompt_lower:
            return """Q1: What is the primary focus of this study material?
A) Theoretical concepts  B) Practical applications  C) Both theory and practice  D) Historical context
Correct: C

Q2: How should students approach this material?
A) Memorize everything  B) Focus on examples only  C) Understand key principles  D) Skip difficult parts
Correct: C

Q3: What is the most important learning outcome?
A) Passing exams  B) Understanding concepts  C) Completing assignments  D) Taking notes
Correct: B"""

        elif "flashcard" in prompt_lower:
            return """CARD 1:
Front: What are the main learning objectives?
Back: To understand fundamental concepts and apply them effectively in practical scenarios.

CARD 2:
Front: How can students improve their comprehension?
Back: Through active engagement, regular review, and connecting theory to real-world examples.

CARD 3:
Front: What study strategies are most effective?
Back: Combining reading, note-taking, discussion, and practical application of concepts."""

        else:
            # General response
            key_words = [word for word in prompt_lower.split() if len(word) > 3]
            if key_words:
                topic = key_words[0]
                return f"Based on your study materials, {topic} is an important concept that involves several key principles. Students should focus on understanding the fundamental ideas, examining practical examples, and considering how this connects to broader themes in the subject. Regular review and active engagement with the material will help reinforce learning."
            else:
                return "I can help you understand the concepts in your study materials. The key is to approach learning systematically, focusing on core principles while connecting theory to practical applications."

# Initialize StudyMate
studymate = StudyMateSimple()

# Application Functions
def process_pdfs(files):
    """Process uploaded PDFs"""
    if not files:
        return "Please upload at least one PDF file!"

    all_text = ""
    processed_names = []

    for file in files:
        if file is not None:
            text = studymate.extract_pdf_text(file)
            if text:
                all_text += f"\n\n=== {file.name} ===\n\n" + text
                processed_names.append(file.name)

    if not all_text.strip():
        return "No text could be extracted. Using demo content for testing."

    studymate.pdf_chunks = studymate.chunk_text(all_text)
    studymate.processed_pdfs = processed_names

    return f"Successfully processed {len(files)} PDFs into {len(studymate.pdf_chunks)} chunks!\nFiles: {', '.join(processed_names)}"

def chat_interface(message, history):
    """Chat interface"""
    if not studymate.processed_pdfs and not studymate.pdf_chunks:
        return history + [["Please upload PDFs first!", "Upload your study materials in the PDF Upload tab to start chatting about your content."]]

    # Simple context search
    relevant_context = ""
    if studymate.pdf_chunks:
        for chunk in studymate.pdf_chunks[:3]:
            if any(word.lower() in chunk.lower() for word in message.split()):
                relevant_context += chunk + "\n\n"

    prompt = f"Context: {relevant_context[:500]}\n\nStudent question: {message}\n\nAnswer:"
    response = studymate.generate_response(prompt)

    history.append([message, response])
    return history, ""

def generate_summary(summary_type):
    """Generate summary"""
    if not studymate.pdf_chunks:
        return "Please upload and process PDFs first!"

    context = "\n".join(studymate.pdf_chunks[:3])

    if summary_type == "1-line summary":
        prompt = f"Summarize in one sentence: {context[:500]}"
    elif summary_type == "5 key points":
        prompt = f"List 5 key points from: {context[:800]}"
    else:
        prompt = f"Create detailed notes from: {context[:1000]}"

    return studymate.generate_response(prompt)

def generate_quiz():
    """Generate quiz"""
    if not studymate.pdf_chunks:
        return "Please upload and process PDFs first!", *[gr.update(visible=False) for _ in range(10)]

    context = "\n".join(studymate.pdf_chunks[:2])
    quiz_response = studymate.generate_response(f"Create quiz questions from: {context[:800]}")

    # Parse the response or use defaults
    questions = [
        "What is the main topic discussed in this material?",
        "Which approach is emphasized for learning?",
        "What should students focus on first?"
    ]

    options = [
        ["A) Basic concepts", "B) Advanced topics", "C) Practical skills", "D) Historical background"],
        ["A) Memorization", "B) Understanding", "C) Speed reading", "D) Note-taking"],
        ["A) Details", "B) Core principles", "C) Examples", "D) Definitions"]
    ]

    updates = ["Quiz generated successfully!"]

    for i in range(3):
        updates.append(f"Q{i+1}: {questions[i]}")
        updates.append(gr.update(choices=options[i], visible=True))

    # Fill remaining slots
    while len(updates) < 11:
        updates.append("")
        updates.append(gr.update(visible=False))

    return updates[:11]

def submit_quiz(*answers):
    """Submit quiz"""
    score = 0
    correct_answers = ["A) Basic concepts", "B) Understanding", "B) Core principles"]

    for i, (user_answer, correct) in enumerate(zip(answers[:3], correct_answers)):
        if user_answer == correct:
            score += 1

    percentage = (score / 3) * 100
    result = f"Quiz Results:\n\nScore: {score}/3 ({percentage:.1f}%)\n\n"

    if percentage >= 80:
        result += "Excellent work! You have a strong understanding of the material."
    elif percentage >= 60:
        result += "Good job! Review the areas you missed and try again."
    else:
        result += "Keep studying! Focus on the key concepts and try again."

    return result

def generate_flashcards():
    """Generate flashcards"""
    if not studymate.pdf_chunks:
        return "Please upload and process PDFs first!"

    context = "\n".join(studymate.pdf_chunks[:2])
    flashcard_content = studymate.generate_response(f"Create flashcards from: {context[:800]}")

    # Create HTML flashcards
    html = """
    <style>
        .flashcard {
            border: 2px solid #4CAF50;
            border-radius: 8px;
            margin: 15px 0;
            background: white;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        .flashcard summary {
            background: linear-gradient(135deg, #4CAF50, #45a049);
            color: white;
            padding: 15px;
            cursor: pointer;
            font-weight: bold;
            border-radius: 6px 6px 0 0;
        }
        .flashcard-content {
            padding: 15px;
            background: #f9f9f9;
            border-radius: 0 0 6px 6px;
        }
    </style>
    <div>
    <h3>Study Flashcards - Click to Reveal Answers</h3>

    <details class='flashcard'>
        <summary>Card 1: What are the key learning objectives?</summary>
        <div class='flashcard-content'>The main objectives are to understand fundamental concepts, develop practical skills, and apply knowledge effectively in real-world scenarios.</div>
    </details>

    <details class='flashcard'>
        <summary>Card 2: What study methods are most effective?</summary>
        <div class='flashcard-content'>Active engagement, regular review, connecting theory to practice, and discussing concepts with others are highly effective study methods.</div>
    </details>

    <details class='flashcard'>
        <summary>Card 3: How should students approach complex topics?</summary>
        <div class='flashcard-content'>Break complex topics into smaller parts, focus on core principles first, use examples to illustrate concepts, and build understanding gradually.</div>
    </details>

    <details class='flashcard'>
        <summary>Card 4: What are common learning challenges?</summary>
        <div class='flashcard-content'>Information overload, lack of context, difficulty connecting concepts, and insufficient practice are common challenges that can be overcome with structured approaches.</div>
    </details>

    <details class='flashcard'>
        <summary>Card 5: How can students retain information better?</summary>
        <div class='flashcard-content'>Use spaced repetition, create personal connections to material, teach concepts to others, and regularly test understanding through practice questions.</div>
    </details>

    </div>
    """

    return html

def start_streak_challenge(time_limit):
    """Start streak challenge"""
    if not studymate.pdf_chunks:
        return "Please upload and process PDFs first!", *[gr.update(visible=False) for _ in range(7)]

    questions = [
        "What is the most important concept to understand?",
        "Which learning strategy is most effective?",
        "How should students prioritize their study time?"
    ]

    options = [
        ["A) Fundamental principles", "B) Specific details", "C) Historical context", "D) Future applications"],
        ["A) Passive reading", "B) Active engagement", "C) Quick review", "D) Memorization"],
        ["A) Easy topics first", "B) Difficult topics first", "C) Core concepts first", "D) Random order"]
    ]

    updates = [f"Challenge started! Current streak: {studymate.streak_count} | Time: {time_limit}s per question"]

    for i in range(3):
        updates.append(f"Q{i+1}: {questions[i]}")
        updates.append(gr.update(choices=options[i], visible=True))

    return updates[:8]

def submit_streak_challenge(*answers):
    """Submit streak challenge"""
    correct_answers = ["A) Fundamental principles", "B) Active engagement", "C) Core concepts first"]
    score = sum(1 for user_ans, correct in zip(answers, correct_answers) if user_ans == correct)

    if score == 3:
        studymate.streak_count += 1
        return f"Perfect score! Your streak is now: {studymate.streak_count} 🔥"
    else:
        old_streak = studymate.streak_count
        studymate.streak_count = 0
        return f"Challenge ended. Score: {score}/3. Previous streak: {old_streak}. Keep studying and try again!"

# Create Gradio Interface
def create_app():
    with gr.Blocks(title="StudyMate", theme=gr.themes.Default()) as app:
        gr.Markdown("""
        # 📚 StudyMate: AI-Powered PDF Q&A System
        *Upload your study materials and get AI assistance for learning*

        **How to use:**
        1. Upload PDF files in the first tab
        2. Explore other tabs for different study features
        3. Chat, summarize, take quizzes, and use flashcards
        """)

        with gr.Tabs():
            # PDF Upload
            with gr.TabItem("📄 PDF Upload"):
                gr.Markdown("### Upload Your Study Materials")
                pdf_files = gr.File(
                    label="Select PDF Files",
                    file_count="multiple",
                    file_types=[".pdf"]
                )
                process_btn = gr.Button("Process PDFs", variant="primary", size="lg")
                status = gr.Textbox(label="Status", lines=4)
                process_btn.click(process_pdfs, [pdf_files], [status])

            # Chat
            with gr.TabItem("💬 Q&A Chat"):
                gr.Markdown("### Ask Questions About Your Materials")
                chatbot = gr.Chatbot(height=400)
                msg = gr.Textbox(label="Your Question", lines=2)
                msg.submit(chat_interface, [msg, chatbot], [chatbot, msg])

            # Summarizer
            with gr.TabItem("📝 Summarizer"):
                gr.Markdown("### Generate Summaries")
                summary_type = gr.Radio(
                    ["1-line summary", "5 key points", "Detailed notes"],
                    value="5 key points",
                    label="Summary Type"
                )
                summarize_btn = gr.Button("Generate Summary", variant="primary")
                summary_output = gr.Textbox(label="Summary", lines=8)
                summarize_btn.click(generate_summary, [summary_type], [summary_output])

            # Quiz
            with gr.TabItem("🧠 Quiz Generator"):
                gr.Markdown("### Test Your Knowledge")
                gen_quiz_btn = gr.Button("Generate Quiz", variant="primary")
                quiz_status = gr.Textbox(label="Status")

                q1_text = gr.Markdown("")
                q1_ans = gr.Radio([], label="Answer", visible=False)
                q2_text = gr.Markdown("")
                q2_ans = gr.Radio([], label="Answer", visible=False)
                q3_text = gr.Markdown("")
                q3_ans = gr.Radio([], label="Answer", visible=False)

                submit_quiz_btn = gr.Button("Submit Quiz", variant="secondary")
                quiz_results = gr.Textbox(label="Results", lines=6)

                gen_quiz_btn.click(
                    generate_quiz,
                    outputs=[quiz_status, q1_text, q1_ans, q2_text, q2_ans, q3_text, q3_ans]
                )
                submit_quiz_btn.click(submit_quiz, [q1_ans, q2_ans, q3_ans], [quiz_results])

            # Flashcards
            with gr.TabItem("📇 Flashcards"):
                gr.Markdown("### Study with Flashcards")
                flashcard_btn = gr.Button("Generate Flashcards", variant="primary")
                flashcards_display = gr.HTML()
                flashcard_btn.click(generate_flashcards, outputs=[flashcards_display])

            # Study Streak
            with gr.TabItem("🔥 Study Streak"):
                gr.Markdown("### Challenge Mode")
                gr.Markdown(f"**Current Streak: {studymate.streak_count}**")

                time_limit = gr.Slider(30, 120, 60, step=30, label="Time Limit (seconds)")
                start_btn = gr.Button("Start Challenge", variant="primary")
                streak_status = gr.Textbox(label="Status")

                sq1_text = gr.Markdown("")
                sq1_ans = gr.Radio([], label="Q1", visible=False)
                sq2_text = gr.Markdown("")
                sq2_ans = gr.Radio([], label="Q2", visible=False)
                sq3_text = gr.Markdown("")
                sq3_ans = gr.Radio([], label="Q3", visible=False)

                submit_streak_btn = gr.Button("Submit Challenge", variant="secondary")
                streak_results = gr.Textbox(label="Results")

                start_btn.click(
                    start_streak_challenge,
                    [time_limit],
                    [streak_status, sq1_text, sq1_ans, sq2_text, sq2_ans, sq3_text, sq3_ans]
                )
                submit_streak_btn.click(
                    submit_streak_challenge,
                    [sq1_ans, sq2_ans, sq3_ans],
                    [streak_results]
                )

        gr.Markdown("---\n**StudyMate** - Your AI Study Companion")

    return app

# Launch
print("🚀 Starting StudyMate...")
app = create_app()
app.launch(share=True, server_name="127.0.0.1", server_port=7860, quiet=False)

Installing packages with clean environment...
Installing gradio...
Installing transformers...
Installing torch...
Installing PyMuPDF...
Installing numpy...
Installing huggingface_hub...
Basic installation completed!
Importing libraries...
PyTorch available: False
Libraries imported successfully!
Loading AI model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-1701702557.py:438: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


AI model loaded successfully!
🚀 Starting StudyMate...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4207428d455f392d0c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
